# Recommendations from Keywords

The main problem of the recommendation systems is cold-start problem. Specifically, it concerns the issue that the system cannot draw any inferences for users or items about which it has not yet gathered sufficient information. For my case, it creates problem when the user or book is new for system. To handle this, I decided to try another approach. I will make recommendations with a few information about user. I will find suitable books from keywords of user such that 'I love to love story' or 'best ever book' etc. To do this, I will find the text similarities of keywords and reviews. It is very common from summaries, genres or names of books to do this, but for a different approach I assumed that I will no information about summaries or genres about books or as general, we can assume that we only have reviews about products from previous customers and we want to offer new customer according to his/her keywords. 

Future improvements can be found at the end of this notebook.

In [1]:
# necessary imports
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
import re
import math
from collections import Counter
import operator

Before begining to set recommendation systems, I would like to make sure about stop words and clean them.

In [2]:
df = pd.read_csv('df_1000.csv') # taking small set

In [66]:
df_full = pd.read_csv('df_2000.csv') # taking larger set

In [3]:
stop = set(stopwords.words("english")) #setting stopwords

In [4]:
# taking only words which are not stopwords
df['word_without_stop'] = df['review_clean'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [67]:
# taking only words which are not stopwords for larger set
df_full['word_without_stop'] = df_full['review_clean'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

For this part of recommendations, I got idea from here;

https://towardsdatascience.com/recommender-system-application-development-part-1-of-4-cosine-similarity-f6dbcd768e83
    
Especially, I took cosine similarity class from there and changed it to function according to my project.

In [5]:
def cosine_similarity_of(text1, text2):
        
        # taking words
        first = re.compile(r"[\w']+").findall(text1)
        second = re.compile(r"[\w']+").findall(text2)

        # creating dictionary from each word and count
        vector1 = Counter(first)
        vector2 = Counter(second)

        # changing vectors to sets to find comman words with intersection
        common = set(vector1.keys()).intersection(set(vector2.keys()))

        dot_product = 0.0

        for i in common:
            # getting amount of each common word for vectors and multiply them also add them
            dot_product += vector1[i] * vector2[i]

        squared_sum_vector1 = 0.0
        squared_sum_vector2 = 0.0

        # getting summation of word counts for every vector
        for i in vector1.keys():
            squared_sum_vector1 += vector1[i]**2

        for i in vector2.keys():
            squared_sum_vector2 += vector2[i]**2

        # calculating magnitude of vector with squared sums
        magnitude = math.sqrt(squared_sum_vector1) * math.sqrt(squared_sum_vector2)

        if not magnitude:
            return 0.0
        else:
            return float(dot_product) / magnitude

In [6]:
text1 = df['word_without_stop'][1] # to check the function taking text 1

In [7]:
text2 = df['word_without_stop'][2] # to check the function taking text 2

In [8]:
cosine_similarity_of(text1, text2)

0.13888386569176292

In [9]:
def get_recommendations(df,keywords):

        score_dict = {}

        for index, row in df.iterrows():
            score_dict[index] = cosine_similarity_of(row['word_without_stop'], keywords)

        # sorting books according to text similarities
        sorted_scores = sorted(score_dict.items(), key=operator.itemgetter(1), reverse=True)

        counter = 0

        # building new df to keep values
        result = pd.DataFrame(columns=('reviewerID', 'asin', 'title','word_without_stop', 'score'))

        # taking highest 10 scored books
        for i in sorted_scores:
            result = result.append({'reviewerID': df.iloc[i[0]]['reviewerID'], 
                                        'asin': df.iloc[i[0]]['asin'], 'title': df.iloc[i[0]]['title'],
                                        'word_without_stop': df.iloc[i[0]]['word_without_stop'], 
                                        'score': i[1]}, ignore_index=True)
            counter += 1

            if counter>9: #setting top-10/ can change according to wanted recommendation number
                break

        return result

In [10]:
keywords = "best book ever" # defining keywords

In [11]:
top_10= get_recommendations(df,keywords)
top_10

,reviewerID,asin,title,word_without_stop,score
0,A1PIHRVCTCJQFL,B018RKHYMQ,The Stocking Was Hung (The Holidays #1) - Kind...,great christmas best years day picked book bes...,0.544331
1,A3CMIEYL0TJLC2,B01D9JB7DW,The Alien King&#39;s Baby: Sci-fi Alien Romanc...,best book wonderful time reading new author go...,0.522233
2,A1O2LQUYDVPOCJ,B00VPAHRLO,The Soccer Mom&#39;s Bad Boy - Kindle edition,wonderful always jordan silver best sexiest wr...,0.518476
3,A3CMIEYL0TJLC2,B01FIGE8YE,Beauty and the Blitz,good book far ready read next one soon best book,0.500000
4,A3T5H007CVA99X,B00FR30D16,Her Best Match: A Clean Billionaire Romance (T...,saw book best romance book read like romance c...,0.500000
5,A3PYB2CTKK93MG,B00EV9LSJI,Tears of Tess (Monsters in the Dark Book 1) - ...,book one best dark romances ever read love gen...,0.489898
6,A1L9WQBSQ5SEFE,B0066HBLUO,CAPTURING THE COWBOY&#39;S HEART - Kindle edition,loved book enjoyable read im going let go book...,0.473879
7,AZYERRDY2VW61,B00629ZTOU,Texas Secrets: The Gallaghers of Morning Star ...,love book super book,0.471405
8,A15EMPO8TCH2RB,B01D4RY4B6,You Don&#39;t Own Me: A Bad Boy Mafia Romance ...,book made cry laughed book great book get two ...,0.471405
9,A19LXU1TT72OKX,B01FRQ4Z8E,Dietrich (Bear Shifter Dating Agency Romance) ...,received arc honest review best book read writ...,0.471405


In [104]:
top_10= get_recommendations(df_full,keywords)

In [105]:
top_10

,reviewerID,asin,title,word_without_stop,score
0,A3CMIEYL0TJLC2,B015QBP4XU,Werewolf Romance: Seduced By The Alpha Wolf Bo...,best book ever read looking great werewolf boo...,0.666667
1,A3CMIEYL0TJLC2,B01B52FWAY,THAT MAN TRILOGY: A Sexy Romantic Comedy - Kin...,finally get read book best book,0.612372
2,A1PIHRVCTCJQFL,B018RKHYMQ,The Stocking Was Hung (The Holidays #1) - Kind...,great christmas best years day picked book bes...,0.544331
3,A2EZZ9UTFNNKRE,B00ZO50HZO,Hot &amp; Bothered (A Hostile Operations Team ...,oh gosh best series ever loved book much rest ...,0.544331
4,A3CMIEYL0TJLC2,B01D9JB7DW,The Alien King&#39;s Baby: Sci-fi Alien Romanc...,best book wonderful time reading new author go...,0.522233
5,A1O2LQUYDVPOCJ,B00VPAHRLO,The Soccer Mom&#39;s Bad Boy - Kindle edition,wonderful always jordan silver best sexiest wr...,0.518476
6,A3CMIEYL0TJLC2,B01FIGE8YE,Beauty and the Blitz,good book far ready read next one soon best book,0.500000
7,A3T5H007CVA99X,B00FR30D16,Her Best Match: A Clean Billionaire Romance (T...,saw book best romance book read like romance c...,0.500000
8,A3MQ4814P5LBTG,B018PPWK0E,The Way Back To Me (The Way Book 1) - Kindle e...,cant find words days completing book still can...,0.495074
9,A30TZBEYTQK723,B01A1E8WO0,Big Rock (Big Rock Book 1) - Kindle edition,book ok wasnt originality book like book invol...,0.493264


I got recommendations for small and larger data set both. As expected, score is increasing when the books and user data are increased. It shows that if I can use this function in whole data set, I can get better results.

# Rating Effect Adding

My system recommends books according to cosine similarity of texts but rating of book is an important parameter for recommendation. So, I will add rating effect to my function. To do this, I will determine how much I want to effect my results. I will use simple math equation; 

$weight = (2*Q/5) * rating - Q$ 

where; Q is set by us according to how much effect will add such as;

if we set Q=10, it increases our cosine similarity score 10%. Some examples can be found as;

In [42]:
# df_effect

,rating,Q,percentage %
0,1,10,-6
1,2,10,-2
2,3,10,2
3,4,10,6
4,5,10,10
5,1,100,-60
6,2,100,-20
7,3,100,20
8,4,100,60
9,5,100,100


I calculated percentage effects to show as an example and only insert here the table. It means that if we set Q values to 10, it increases our cosine similarity value 10% when rating is 5. If we set it to 100, it increases cosine similarity value 100% when rating is 5. Also, it decreases cosine similarity value if the rating is lower than 3.

In [44]:
def rating_weight(rating,Q):
    '''calculating rating weight'''
    w = (2*Q/5)*rating - Q
    return w

In [45]:
def calculate_final_score(cos_sim, w):
        '''adding weight effect to cosine similarity'''
        effect = (cos_sim / 100) * w
        return cos_sim + effect

In [46]:
def get_recommendations_rat(df,keywords):
        '''getting recommendations with rating effect'''
        score_dict = {}

        for index, row in df.iterrows():
            cs_score = cosine_similarity_of(row['word_without_stop'], keywords)

            rating = row['overall']
            
            # I choose Q as 10 to not to change weight too much
            rating_effect = rating_weight(rating,10)

            last_score = calculate_final_score(cs_score, rating_effect)

            score_dict[index] = last_score
            
        # sorting books according to text similarities
        sorted_scores = sorted(score_dict.items(), key=operator.itemgetter(1), reverse=True)

        counter = 0

        # building new df to keep values
        result = pd.DataFrame(columns=('reviewerID', 'asin','title','word_without_stop','overall', 'score'))

         # taking highest 10 scored books
        for i in sorted_scores:
        
            result = result.append({'reviewerID': df.iloc[i[0]]['reviewerID'], 'asin': df.iloc[i[0]]['asin'],
                                        'title': df.iloc[i[0]]['title'],
                                        'word_without_stop': df.iloc[i[0]]['word_without_stop'],
                                        'overall': df.iloc[i[0]]['overall'],
                                        'score': i[1]}, ignore_index=True)
            counter += 1

            if counter>9:
                break
                
        return result

In [47]:
keywords = "best book ever"

In [48]:
top_10= get_recommendations_rat(df,keywords) # for small subset

In [49]:
top_10

,reviewerID,asin,title,word_without_stop,overall,score
0,A1PIHRVCTCJQFL,B018RKHYMQ,The Stocking Was Hung (The Holidays #1) - Kind...,great christmas best years day picked book bes...,5,0.598764
1,A3CMIEYL0TJLC2,B01D9JB7DW,The Alien King&#39;s Baby: Sci-fi Alien Romanc...,best book wonderful time reading new author go...,5,0.574456
2,A1O2LQUYDVPOCJ,B00VPAHRLO,The Soccer Mom&#39;s Bad Boy - Kindle edition,wonderful always jordan silver best sexiest wr...,5,0.570323
3,A3T5H007CVA99X,B00FR30D16,Her Best Match: A Clean Billionaire Romance (T...,saw book best romance book read like romance c...,5,0.550000
4,A3PYB2CTKK93MG,B00EV9LSJI,Tears of Tess (Monsters in the Dark Book 1) - ...,book one best dark romances ever read love gen...,5,0.538888
5,A3CMIEYL0TJLC2,B01FIGE8YE,Beauty and the Blitz,good book far ready read next one soon best book,4,0.530000
6,A1L9WQBSQ5SEFE,B0066HBLUO,CAPTURING THE COWBOY&#39;S HEART - Kindle edition,loved book enjoyable read im going let go book...,5,0.521267
7,AZYERRDY2VW61,B00629ZTOU,Texas Secrets: The Gallaghers of Morning Star ...,love book super book,5,0.518545
8,A15EMPO8TCH2RB,B01D4RY4B6,You Don&#39;t Own Me: A Bad Boy Mafia Romance ...,book made cry laughed book great book get two ...,5,0.518545
9,A19LXU1TT72OKX,B01FRQ4Z8E,Dietrich (Bear Shifter Dating Agency Romance) ...,received arc honest review best book read writ...,5,0.518545


In [75]:
top_10= get_recommendations_rat(df_full,keywords) #for large subset
top_10

,reviewerID,asin,title,word_without_stop,overall,score
0,A3CMIEYL0TJLC2,B015QBP4XU,Werewolf Romance: Seduced By The Alpha Wolf Bo...,best book ever read looking great werewolf boo...,5,0.733333
1,A3CMIEYL0TJLC2,B01B52FWAY,THAT MAN TRILOGY: A Sexy Romantic Comedy - Kin...,finally get read book best book,5,0.673610
2,A1PIHRVCTCJQFL,B018RKHYMQ,The Stocking Was Hung (The Holidays #1) - Kind...,great christmas best years day picked book bes...,5,0.598764
3,A2EZZ9UTFNNKRE,B00ZO50HZO,Hot &amp; Bothered (A Hostile Operations Team ...,oh gosh best series ever loved book much rest ...,5,0.598764
4,A3CMIEYL0TJLC2,B01D9JB7DW,The Alien King&#39;s Baby: Sci-fi Alien Romanc...,best book wonderful time reading new author go...,5,0.574456
5,A1O2LQUYDVPOCJ,B00VPAHRLO,The Soccer Mom&#39;s Bad Boy - Kindle edition,wonderful always jordan silver best sexiest wr...,5,0.570323
6,A3T5H007CVA99X,B00FR30D16,Her Best Match: A Clean Billionaire Romance (T...,saw book best romance book read like romance c...,5,0.550000
7,A3MQ4814P5LBTG,B018PPWK0E,The Way Back To Me (The Way Book 1) - Kindle e...,cant find words days completing book still can...,5,0.544581
8,A2F5K1M79GW649,B01ACB72TS,Brothers Black: Wyatt the Heartbreaker - Kindl...,book book world great man like wyattthis book ...,5,0.540089
9,A3PYB2CTKK93MG,B00EV9LSJI,Tears of Tess (Monsters in the Dark Book 1) - ...,book one best dark romances ever read love gen...,5,0.538888


When we compare our scores with previous system, it is obviously seen that score increased because we added rating effect on it.

# Number of Ratings Effect

It is hard to say that the book which has average rating as 5 with only 8 comment is better than the book rated as 4 with 2000 reviews. More reviews on product means more reliability. So, I will add multiplier to my rating function which give less effect if the rating is less than threshold. I will determine a threshold rating value for this. If the number of rating is not above this threshold, it will not effect my function too much.

PS: I have already chosen my sample data above the threshold values, so if I will not see too much difference it is very normal. The difference will be high, if I will use whole dataset.

To do this, I will do very simple math again;

$multiplier = e^{-t*0,68/rating number} $

where t is threshold values which I set. We can choose another value different than e number but multiplier must be 0.5 when threshold is equal to the total rating number of the book. According to this equation, my multiplier will be between 0 and 1.

In [50]:
math.exp(-0.68) # when threshold = total rating number of book

0.5066169923655895

In [61]:
def get_rating_weight_with_threshold(rating, count, threshold, Q):
            # rating effect 
            w = (2*Q/5)*rating - Q
            # multiplier effect
            M = math.exp((-threshold*0.68)/count)

            return w * M

In [62]:
def get_recommendations_rat_count(df,keywords):

        score_dict = {}

        for index, row in df.iterrows():
            cs = cosine_similarity_of(row['word_without_stop'], keywords)

            rating = row['overall']
            rating_count = row['Book_Count']
            threshold = 110
            # I choose Q as 10 to not to change weight too much
            rating_contribution = get_rating_weight_with_threshold(rating,rating_count,threshold,10)

            last_score = calculate_final_score(cs, rating_contribution)

            score_dict[index] = last_score
            
        # sorting books according to text similarities
        sorted_scores = sorted(score_dict.items(), key=operator.itemgetter(1), reverse=True)

        counter = 0

        # building new df to keep values
        result = pd.DataFrame(columns=('reviewerID', 'asin','title','word_without_stop','overall', 'score'))

        # taking highest 10 scored books
        for i in sorted_scores:
            
            result = result.append({'reviewerID': df.iloc[i[0]]['reviewerID'], 'asin': df.iloc[i[0]]['asin'],
                                        'title': df.iloc[i[0]]['title'],
                                        'word_without_stop': df.iloc[i[0]]['word_without_stop'],
                                        'overall': df.iloc[i[0]]['overall'],
                                        'score': i[1]}, ignore_index=True)
            counter += 1

            if counter>9:
                break
                
        return result

In [64]:
keywords = "best book ever"
top_10= get_recommendations_rat_count(df,keywords)
top_10

,reviewerID,asin,title,word_without_stop,overall,score
0,A1PIHRVCTCJQFL,B018RKHYMQ,The Stocking Was Hung (The Holidays #1) - Kind...,great christmas best years day picked book bes...,5,0.587226
1,A3CMIEYL0TJLC2,B01D9JB7DW,The Alien King&#39;s Baby: Sci-fi Alien Romanc...,best book wonderful time reading new author go...,5,0.556857
2,A1O2LQUYDVPOCJ,B00VPAHRLO,The Soccer Mom&#39;s Bad Boy - Kindle edition,wonderful always jordan silver best sexiest wr...,5,0.550375
3,A3T5H007CVA99X,B00FR30D16,Her Best Match: A Clean Billionaire Romance (T...,saw book best romance book read like romance c...,5,0.537500
4,A3PYB2CTKK93MG,B00EV9LSJI,Tears of Tess (Monsters in the Dark Book 1) - ...,book one best dark romances ever read love gen...,5,0.534190
5,A3CMIEYL0TJLC2,B01FIGE8YE,Beauty and the Blitz,good book far ready read next one soon best book,4,0.522694
6,AZYERRDY2VW61,B00629ZTOU,Texas Secrets: The Gallaghers of Morning Star ...,love book super book,5,0.510718
7,A1L9WQBSQ5SEFE,B0066HBLUO,CAPTURING THE COWBOY&#39;S HEART - Kindle edition,loved book enjoyable read im going let go book...,5,0.509800
8,A34QXHSIS1IFNS,B00HWKW7K2,Forty 2 Days (Billionaire Banker Series) - Kin...,book best needed great read get,5,0.502149
9,A15EMPO8TCH2RB,B01D4RY4B6,You Don&#39;t Own Me: A Bad Boy Mafia Romance ...,book made cry laughed book great book get two ...,5,0.501112


In [68]:
keywords = "best book ever"
top_10= get_recommendations_rat_count(df_full,keywords)
top_10

,reviewerID,asin,title,word_without_stop,overall,score
0,A3CMIEYL0TJLC2,B015QBP4XU,Werewolf Romance: Seduced By The Alpha Wolf Bo...,best book ever read looking great werewolf boo...,5,0.698455
1,A3CMIEYL0TJLC2,B01B52FWAY,THAT MAN TRILOGY: A Sexy Romantic Comedy - Kin...,finally get read book best book,5,0.646969
2,A1PIHRVCTCJQFL,B018RKHYMQ,The Stocking Was Hung (The Holidays #1) - Kind...,great christmas best years day picked book bes...,5,0.587226
3,A2EZZ9UTFNNKRE,B00ZO50HZO,Hot &amp; Bothered (A Hostile Operations Team ...,oh gosh best series ever loved book much rest ...,5,0.570286
4,A3CMIEYL0TJLC2,B01D9JB7DW,The Alien King&#39;s Baby: Sci-fi Alien Romanc...,best book wonderful time reading new author go...,5,0.556857
5,A1O2LQUYDVPOCJ,B00VPAHRLO,The Soccer Mom&#39;s Bad Boy - Kindle edition,wonderful always jordan silver best sexiest wr...,5,0.550375
6,A3T5H007CVA99X,B00FR30D16,Her Best Match: A Clean Billionaire Romance (T...,saw book best romance book read like romance c...,5,0.537500
7,A3PYB2CTKK93MG,B00EV9LSJI,Tears of Tess (Monsters in the Dark Book 1) - ...,book one best dark romances ever read love gen...,5,0.534190
8,A2F5K1M79GW649,B01ACB72TS,Brothers Black: Wyatt the Heartbreaker - Kindl...,book book world great man like wyattthis book ...,5,0.528285
9,A3MQ4814P5LBTG,B018PPWK0E,The Way Back To Me (The Way Book 1) - Kindle e...,cant find words days completing book still can...,5,0.523521


# Positive Review Effect

Now, I want to give more weight to my score if the review is positive. To do this, I will create new columns which is True when rating is equal or higher than 3.

In [67]:
df['positive'] = df['overall'].apply(lambda x: True if x>=3 else False)

In [80]:
df_full['positive'] = df_full['overall'].apply(lambda x: True if x>=3 else False)

for positive reviews rating is calculated as;

$newrating = rating + (7-rating)/2 = (rating+7)/2$

it means for 3 it will effect our score not as 3:
- for 3 effects as 5
- for 4, effects as 5,5
- for 5 , effect as 6

for negative reviews;

$newrating = rating - rating/2 = rating/2$

- for 2, effects as 1
- for 1, effects as 0,5

With this implemantation, I will make the difference higher between negative and positive reviews. 

In [66]:
def get_recommendations_rat_count_pos(df,keywords):

        score_dict = {}

        for index, row in df.iterrows():
            cs = cosine_similarity_of(row['word_without_stop'], keywords)

            rating = row['overall']
            rating_count = row['Book_Count']
            
            if row['positive']==True:
                pos_rat = (7+rating)/2
            else:
                pos_rat = rating/2
                
            threshold=110
            # I choose Q as 10 to not to change weight too much
            rating_contribution =get_rating_weight_with_threshold(pos_rat,rating_count,threshold,10)

            last_score = calculate_final_score(cs, rating_contribution)
            # keeping scores
            score_dict[index] = last_score
            
        #sort cities by score and index.
        sorted_scores = sorted(score_dict.items(), key=operator.itemgetter(1), reverse=True)

        counter = 0

        # building new df to keep values
        result = pd.DataFrame(columns=('reviewerID', 'asin','title','word_without_stop','overall', 'score'))

        # taking highest 10 scored books
        for i in sorted_scores:
            
            result = result.append({'reviewerID': df.iloc[i[0]]['reviewerID'], 'asin': df.iloc[i[0]]['asin'],
                                        'title': df.iloc[i[0]]['title'],
                                        'word_without_stop': df.iloc[i[0]]['word_without_stop'],
                                        'overall': df.iloc[i[0]]['overall'],
                                        'score': i[1]}, ignore_index=True)
            counter += 1

            if counter>9:
                break
                
        return result

In [68]:
keywords = "best book ever"
top_10= get_recommendations_rat_count_pos(df,keywords)
top_10

,reviewerID,asin,title,word_without_stop,overall,score
0,A1PIHRVCTCJQFL,B018RKHYMQ,The Stocking Was Hung (The Holidays #1) - Kind...,great christmas best years day picked book bes...,5,0.612717
1,A3CMIEYL0TJLC2,B01D9JB7DW,The Alien King&#39;s Baby: Sci-fi Alien Romanc...,best book wonderful time reading new author go...,5,0.582887
2,A1O2LQUYDVPOCJ,B00VPAHRLO,The Soccer Mom&#39;s Bad Boy - Kindle edition,wonderful always jordan silver best sexiest wr...,5,0.576683
3,A3T5H007CVA99X,B00FR30D16,Her Best Match: A Clean Billionaire Romance (T...,saw book best romance book read like romance c...,5,0.561419
4,A3PYB2CTKK93MG,B00EV9LSJI,Tears of Tess (Monsters in the Dark Book 1) - ...,book one best dark romances ever read love gen...,5,0.555412
5,A3CMIEYL0TJLC2,B01FIGE8YE,Beauty and the Blitz,good book far ready read next one soon best book,4,0.552851
6,A1L9WQBSQ5SEFE,B0066HBLUO,CAPTURING THE COWBOY&#39;S HEART - Kindle edition,loved book enjoyable read im going let go book...,5,0.532372
7,AZYERRDY2VW61,B00629ZTOU,Texas Secrets: The Gallaghers of Morning Star ...,love book super book,5,0.532174
8,A34QXHSIS1IFNS,B00HWKW7K2,Forty 2 Days (Billionaire Banker Series) - Kin...,book best needed great read get,5,0.525748
9,A15EMPO8TCH2RB,B01D4RY4B6,You Don&#39;t Own Me: A Bad Boy Mafia Romance ...,book made cry laughed book great book get two ...,5,0.524907


In [82]:
keywords = "best book ever"
top_10= get_recommendations_rat_count_pos(df_full,keywords)
top_10

,reviewerID,asin,title,word_without_stop,overall,score
0,A3CMIEYL0TJLC2,B015QBP4XU,Werewolf Romance: Seduced By The Alpha Wolf Bo...,best book ever read looking great werewolf boo...,5,0.733323
1,A3CMIEYL0TJLC2,B01B52FWAY,THAT MAN TRILOGY: A Sexy Romantic Comedy - Kin...,finally get read book best book,5,0.678507
2,A1PIHRVCTCJQFL,B018RKHYMQ,The Stocking Was Hung (The Holidays #1) - Kind...,great christmas best years day picked book bes...,5,0.612717
3,A2EZZ9UTFNNKRE,B00ZO50HZO,Hot &amp; Bothered (A Hostile Operations Team ...,oh gosh best series ever loved book much rest ...,5,0.598755
4,A3CMIEYL0TJLC2,B01D9JB7DW,The Alien King&#39;s Baby: Sci-fi Alien Romanc...,best book wonderful time reading new author go...,5,0.582887
5,A1O2LQUYDVPOCJ,B00VPAHRLO,The Soccer Mom&#39;s Bad Boy - Kindle edition,wonderful always jordan silver best sexiest wr...,5,0.576683
6,A3T5H007CVA99X,B00FR30D16,Her Best Match: A Clean Billionaire Romance (T...,saw book best romance book read like romance c...,5,0.561419
7,A3PYB2CTKK93MG,B00EV9LSJI,Tears of Tess (Monsters in the Dark Book 1) - ...,book one best dark romances ever read love gen...,5,0.555412
8,A3CMIEYL0TJLC2,B01FIGE8YE,Beauty and the Blitz,good book far ready read next one soon best book,4,0.552851
9,A2F5K1M79GW649,B01ACB72TS,Brothers Black: Wyatt the Heartbreaker - Kindl...,book book world great man like wyattthis book ...,5,0.551652


According to work above, when we add rating effect on function, we get higher scores and it is obviously seen that score is increasing when I use larger data. 

## Future Improvements:

- I will add to positive and negative review number effect such as if the book has 1000 positive review, it will effect score more than the book with 500 positive.
- I will apply this recommendation system for whole data set to get the best result.